<a href="https://colab.research.google.com/github/aSafarpoor/Seminar/blob/main/Embedding_%26_Normaliz_Description.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

source link: http://deepyeti.ucsd.edu/jianmo/amazon/index.html

In [158]:
import json
from tqdm import tqdm
import random
from collections import Counter
import pandas as pd
import pickle
import numpy as np

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
%cd drive
%cd MyDrive
%cd Seminar
!ls

[Errno 2] No such file or directory: 'drive'
/content/drive/MyDrive/Seminar
[Errno 2] No such file or directory: 'MyDrive'
/content/drive/MyDrive/Seminar
[Errno 2] No such file or directory: 'Seminar'
/content/drive/MyDrive/Seminar
AMAZON_FASHION.json	  meta_AMAZON_FASHION.json.gz  user_tweets.p
AMAZON_FASHION.json.gz	  meta_data.p
meta_AMAZON_FASHION.json  user_item_matrix.p


In [ ]:
# !wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/AMAZON_FASHION.json.gz

In [ ]:
# !wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_AMAZON_FASHION.json.gz

In [ ]:
# !ls

In [ ]:
# !gunzip -k AMAZON_FASHION.json.gz

In [ ]:
# !gunzip -k meta_AMAZON_FASHION.json.gz

In [9]:
!ls

AMAZON_FASHION.json	  meta_AMAZON_FASHION.json.gz  user_tweets.p
AMAZON_FASHION.json.gz	  meta_data.p
meta_AMAZON_FASHION.json  user_item_matrix.p


# data fast overview

In [44]:
meta_data = []
for line in open('meta_AMAZON_FASHION.json', 'r'):
    meta_data.append(json.loads(line))

In [45]:
len(meta_data) , '\n' , meta_data[0]

(186637,
 '\n',
 {'asin': '0764443682',
  'brand': 'Group Publishing (CO)',
  'date': '8.70 inches',
  'feature': ['Product Dimensions:\n                    \n8.7 x 3.6 x 11.4 inches',
   'Shipping Weight:\n                    \n2.4 pounds'],
  'imageURL': ['https://images-na.ssl-images-amazon.com/images/I/51bSrINiWpL._US40_.jpg'],
  'imageURLHighRes': ['https://images-na.ssl-images-amazon.com/images/I/51bSrINiWpL.jpg'],
  'rank': '13,052,976inClothing,Shoesamp;Jewelry(',
  'title': 'Slime Time Fall Fest [With CDROM and Collector Cards and Neutron Balls, Incredi-Ball and Glow Stick Necklace, Paper Fram'})

In [46]:
t_count = i_count = hi_count = 0
tlen= []
for i in meta_data:
    if 'title' in list(i.keys()):
        t_count +=1
        tlen.append(len(i['title']))
    if 'imageURL' in list(i.keys()):
        i_count +=1
    if 'imageURLHighRes' in list(i.keys()):
        hi_count +=1

t_count , i_count , hi_count , len(meta_data)

(186632, 132017, 132017, 186637)

In [47]:
md = pd.DataFrame(meta_data)

In [48]:
print(len(md)) , md.columns

186637


(None, Index(['title', 'brand', 'feature', 'rank', 'date', 'asin', 'imageURL',
        'imageURLHighRes', 'description', 'price', 'also_view', 'also_buy',
        'fit', 'details', 'similar_item', 'tech1'],
       dtype='object'))

In [49]:
md_text = md[~md['description'].isnull()]
md_img = md[~md['imageURL'].isnull()]

md_text_image = md_img[~md_img['description'].isnull()]

print(len(md),len(md_img),len(md_text),len(md_text_image))

186637 132017 15869 9810


In [50]:
df = md_text_image
md = md_img = md_text = md_text_image = ''

In [130]:
df = stored_df
# stored_df = df
df.head

<bound method NDFrame.head of                                                     title  ... tech1
2                                   magical things really  ...   NaN
17                   x l carbon fiber money clip made usa  ...   NaN
18      shimmer anne shine clip costumehalloween cat ears  ...   NaN
69                     buxton heiress pikmeup framed case  ...   NaN
70      fossil bifold super capacity wallet chicago bl...  ...   NaN
...                                                   ...  ...   ...
186506  womens sexy gladiator costume halloween costum...  ...   NaN
186538                            georgia boot amp insole  ...   NaN
186552  single flare steel plugs mint green rose frame...  ...   NaN
186557           converse unisex ct star oxford sharkskin  ...   NaN
186589                princeton university tigers pennant  ...   NaN

[9810 rows x 16 columns]>

In [131]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

class Normalizer:
    # based on https://www.geeksforgeeks.org/normalizing-textual-data-with-python/
    def __init__(self):
        # import regex
        self.stop_words = set(stopwords.words('english'))

    def text_normalizer(self,text):
        if type(text) == type([]):
            text = text[0]
        text = str(text)
        # convert to lower case
        lower_string = text.lower()
        # remove numbers
        no_number_string = re.sub(r'\d+','',lower_string)
        # remove all punctuation except words and space
        no_punc_string = re.sub(r'[^\w\s]','', no_number_string)
        # remove white spaces
        no_wspace_string = no_punc_string.strip()
        # convert string to list of words
        lst_string = [no_wspace_string][0].split()
        # remove stopwords
        no_stpwords_string=""
        for i in lst_string:
            if not i in self.stop_words:
                no_stpwords_string += i+' '
        # removing last space
        no_stpwords_string = no_stpwords_string[:-1]
        output = no_stpwords_string
        return output

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [132]:
normalizer = Normalizer()


In [133]:
df["description"] = df["description"].apply(normalizer.text_normalizer)
df["title"] = df["title"].apply(normalizer.text_normalizer)

In [134]:
df.head

<bound method NDFrame.head of                                                     title  ... tech1
2                                   magical things really  ...   NaN
17                   x l carbon fiber money clip made usa  ...   NaN
18      shimmer anne shine clip costumehalloween cat ears  ...   NaN
69                     buxton heiress pikmeup framed case  ...   NaN
70      fossil bifold super capacity wallet chicago bl...  ...   NaN
...                                                   ...  ...   ...
186506  womens sexy gladiator costume halloween costum...  ...   NaN
186538                            georgia boot amp insole  ...   NaN
186552  single flare steel plugs mint green rose frame...  ...   NaN
186557           converse unisex ct star oxford sharkskin  ...   NaN
186589                princeton university tigers pennant  ...   NaN

[9810 rows x 16 columns]>

# embedding vord2vec

In [ ]:
# from gensim.models import Word2Vec

# from gensim.models import Word2Vec
# from sklearn.decomposition import PCA
# from matplotlib import pyplot

# import nltk
# nltk.download('punkt')

# sentences = []
# for text in tqdm(df['description']):
#     tokens = nltk.word_tokenize(text)
#     sentences.append(tokens)

# # train model
# model = Word2Vec(sentences, min_count=1)
# # summarize the loaded model
# print(model)
# # summarize vocabulary
# words = list(model.wv.vocab)

# # save model
# model.save('model.bin')
# # load model
# new_model = Word2Vec.load('model.bin')
# print(new_model)

# X = model[model.wv.vocab]
# pca = PCA(n_components=2)
# result = pca.fit_transform(X)
# # create a scatter plot of the projection
# pyplot.figure(figsize=(15,15))
# pyplot.scatter(result[10:100, 0], result[10:100, 1])
# words = list(model.wv.vocab)

# for i, word in enumerate(words[10:100]):
# 	pyplot.annotate(word, xy=(result[i, 0], result[i, 1]))
# pyplot.show()

# SentenceBert

ref: https://www.analyticsvidhya.com/blog/2020/08/top-4-sentence-embedding-techniques-using-python/

In [135]:
try:
    from sentence_transformers import SentenceTransformer
except:
    ! pip install sentence_transformers
    from sentence_transformers import SentenceTransformer

sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [136]:
def embedding_sentenceBert(sent):
    sent_embedding = sbert_model .encode(sent)
    return sent_embedding

In [154]:
print(len(df))
print(df.columns)

9810
Index(['title', 'brand', 'feature', 'rank', 'date', 'asin', 'imageURL',
       'imageURLHighRes', 'description', 'price', 'also_view', 'also_buy',
       'fit', 'details', 'similar_item', 'tech1', 'embedded_description'],
      dtype='object')


In [171]:
df.drop_duplicates(subset=['title', 'description'],keep=False, inplace=True)

In [172]:
df.reset_index(drop=True, inplace=True)

In [173]:
print(len(df))

8538


In [137]:
tqdm.pandas()
df['embedded_description'] = df['description'].progress_apply(embedding_sentenceBert)

100%|██████████| 9810/9810 [38:16<00:00,  4.27it/s]


In [159]:

def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [166]:
len(aa),len(df),len(bb)

(9638, 9638, 9638)

In [183]:
aa = df['embedded_description']
aa = list(aa)
bb = df['description']
bb = list(bb)
t = to = 0
a = aa[:100]
for i in tqdm(range(len(a))):
    for j in range(i,len(a)):
        to += 1
        sim = cosine(a[i], a[j])
        if sim>0.9 and i != j:
            t+=1
            print('')
            print(sim,i,j)
            print(bb[i])
            print(bb[j])
            print("=======================")
            if t>10:
                break
    if t>10:
        break

 53%|█████▎    | 53/100 [00:00<00:00, 268.27it/s]


0.91037583 2 3
classically designed silver solitaire ring last lifetime expertly crafted ct clear emeraldcut cubic zirconia lays top sterling silver band mm wide ring timeless stack another ring wear alone make entire hand sparkle unbelievable price theres reason ring get ring shipped free gift box perfect surprising special someone
sparkling magnificent sterling silver band measures approximately mm wide featuring elongated clear ct emerald cut cubic zirconia marquise shaped stone beautiful puts real diamonds shame get look real diamonds without hefty price tag ring shipped free gift box perfect surprising special someone

0.94727796 7 8
bracelet silver ball rosary sterling silver mm inch necklace made highest quality metal shipped free pendant box make perfect gift also wear elegantly crafted symbol proudly brilliant exquisite way express anchor faith treasure keepsake years come jewelry comes satisfaction guarantee reassured happy purchase
cross holy spirit k yellow gold xmm inch n

100%|██████████| 100/100 [00:00<00:00, 341.79it/s]


0.912392 58 79
reebok nfl player longsleeve mock perfect layering piece cotton polyester spandex longsleeve mock embroidered appliqu chest piping detail front back nfl shield embroidered center back neck team label lower left side seam reebok logo embroidered left sleeve imported
reebok nfl pastel zip top made cotton polyester fleece team name embroidered left chest imported

1.0 86 87
wellness bracelet unique jewelry piece customized wearer featuring simple sleek design watermelon tourmaline rondells sterling silver piece uses healing properties tourmaline inspiration tourmaline known promote healing ability transform repel negative energies tourmaline thought diminish fears promoting selfunderstanding calming effect believed put one ease allowing look past seriousness situation allowing healing energy take wellness bracelet increases power allowing wearer pick milagro add bracelet bracelet come one milagro attached clasp see amazon storefront view milagros available choose milagro c

In [178]:
df.head

<bound method NDFrame.head of                                                   title  ...                               embedded_description
0                                 magical things really  ...  [-0.5024358, 0.99598145, 0.41040987, 0.1702736...
1                  x l carbon fiber money clip made usa  ...  [-0.25916183, 0.5187903, -0.52674836, 0.148806...
2     sterling silver cubic zirconia solitaire ring ...  ...  [-0.6441098, 0.943525, -0.64295757, 0.50885004...
3     sterling silver cz marquise cut solitaire cubi...  ...  [-0.46400547, 0.83838195, -1.1143519, 0.509472...
4           calvin klein mens pack classic vneck tshirt  ...  [0.37892506, -0.53210634, 0.10035399, 0.423677...
...                                                 ...  ...                                                ...
8533  womens sexy gladiator costume halloween costum...  ...  [-0.7679501, 0.35930666, 0.9350776, 0.5442182,...
8534                            georgia boot amp insole  ...  [-0.5304945,

In [ ]:
# import pickle
# with open('df_with_embedded_text.p', 'wb') as fp:
#     pickle.dump(df, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [179]:
print(df.iloc[86])

title                         tourmaline wellness braceletsleeping person
brand                                                     Milagro Jewelry
feature                                                               NaN
rank                                   26,187,916inClothing,ShoesJewelry(
date                                                      Milagro Jewelry
asin                                                           B000BTDJY4
imageURL                [https://images-na.ssl-images-amazon.com/image...
imageURLHighRes         [https://images-na.ssl-images-amazon.com/image...
description             wellness bracelet unique jewelry piece customi...
price                                                                 NaN
also_view                                                             NaN
also_buy                                                              NaN
fit                                                                   NaN
details                               

In [180]:
print(df.iloc[87])

title                                   tourmaline wellness braceletheart
brand                                                     Milagro Jewelry
feature                                                               NaN
rank                                   25,759,325inClothing,ShoesJewelry(
date                                                      Milagro Jewelry
asin                                                           B000BTDJYE
imageURL                [https://images-na.ssl-images-amazon.com/image...
imageURLHighRes         [https://images-na.ssl-images-amazon.com/image...
description             wellness bracelet unique jewelry piece customi...
price                                                                 NaN
also_view                                                             NaN
also_buy                                                              NaN
fit                                                                   NaN
details                               

In [182]:
bb[86]==bb[87]

True

# save clean dataframe from users:

In [184]:
import pickle
with open('embedded_text.p', 'wb') as fp:
    pickle.dump(df, fp, protocol=pickle.HIGHEST_PROTOCOL)

"\nwith open('data.p', 'rb') as fp:\n    data = pickle.load(fp)\n"

In [185]:
# with open('embedded_text.p', 'rb') as fp:
#     data = pickle.load(fp)